In [ ]:
import numpy as np
from scipy.fftpack import fft, ifft, fftfreq, hilbert
from scipy import signal
from scipy.signal import hilbert as analytic
import matplotlib.pyplot as plt
import mpld3
from mpld3 import plugins
mpl.style.use('monokai')
#mpl.style.use('default')
#mpl.rcParams['text.color'] = 'black'

In [ ]:
fc = 40e3
f0 = 1e3
fs = 1e6

db = lambda x: 20*np.log10(np.abs(x))
periods = 100
nsamples = np.ceil(periods *fs/f0).astype('int')
t=np.arange(nsamples)/fs
f = fftfreq(nsamples,1/fs)
carrier = np.exp(1j*2*np.pi*fc*t)
audio = np.sin(2*np.pi*f0*t)#+0.7*np.sin(4*np.pi*f0*t)+0.5*np.sin(6*np.pi*f0*t)+0.3*np.sin(8*np.pi*f0*t)+0.1*np.sin(10*np.pi*f0*t)
audio /= np.max(np.abs(audio))

Normal AM modulation gives a reproducable transducer output, but the demodulated signal will contain harmonics.

In [ ]:
envelope = 1+0.95*audio
modulated_simple = envelope*carrier.real
modulated_simple_spectrum = fft(modulated_simple)/nsamples
demodulated_simple = np.abs(analytic(modulated_simple))**2
demodulated_simple_spectrum = fft(demodulated_simple)/nsamples

plt.plot(f,db(modulated_simple_spectrum))
plt.xlim((30e3, 50e3))
plt.ylim((-60,0))
plt.show()
plt.plot(f,db(demodulated_simple_spectrum))
plt.xlim((0,5e3))
plt.ylim((-60,5))
plt.show()

Using a square root in the envelope calculation will in theory give a perfect demodulated output.
However, the trancducer output will contain too many harmonics to reproduce. If this is simulated with a simple bandpass filter we can see the harmonics in the demodulated output.

In [ ]:
envelope = np.sqrt(1+0.95*audio)
modulated_sqrt = envelope*carrier.real
bpf = signal.iirfilter(2,[39e3/fs*2, 41e3/fs*2],output='sos')
modulated_sqrt = signal.sosfilt(bpf, modulated_sqrt)
modulated_sqrt_spectrum = fft(modulated_sqrt)/nsamples
demodulated_sqrt = np.abs(analytic(modulated_sqrt))**2
demodulated_sqrt_spectrum = fft(demodulated_sqrt)/nsamples

plt.plot(f,db(modulated_sqrt_spectrum))
plt.xlim((30e3, 50e3))
plt.ylim((-60,0))
plt.show()
plt.plot(f,db(demodulated_sqrt_spectrum))
plt.xlim((0,5e3))
plt.ylim((-60,0))
plt.show()

If we create the analytic signal of the envelope as 
$$m_a(t) = m(t) + j\mathcal{H}\{m(t)\} $$
we will obtain only sidebands on one side of the carrier, but the demodulated output is not good anymore.

In [ ]:
modulated_analytic = np.real(analytic(envelope) * carrier)
modulated_analytic_spectrum = fft(modulated_analytic)/nsamples
demodulated_analytic = np.abs(analytic(modulated_analytic))**2
demodulated_analytic_spectrum = fft(demodulated_analytic)/nsamples

plt.plot(f,db(modulated_analytic_spectrum))
plt.xlim((30e3, 50e3))
plt.ylim((-60,0))
plt.show()
plt.plot(f,db(demodulated_analytic_spectrum))
plt.xlim((0,5e3))
plt.ylim((-60,0))
plt.show()

If we instead study the case when the modulation has been moved to the exponent, as
$$ y(t) = e^{\log(m(t))} e^{j\omega t} $$
we cad add a pure phase shift as 
$$ y(t) = e^{\log(m(t))} e^{j\phi(t)} e^{j\omega t} $$
If the phase shift $\phi$ is chosen to create an analytic signal, i.e.
$$ \phi(t) = \mathcal{H}\{\log(m(t))\} $$
we see that the new modulation is given by
$$ \tilde m = e^{ \log(m) +j\mathcal{H}\{\log(m)\} }  = m(t) e^{j\mathcal{H}\{\log(m(t))\} } $$
This technique was first introduced by Powers (1960) and explaned in an overview by Bedrosian (1962) as exponential or envelope modulation.
If we define $g(t) = \alpha \log f(t) $ where $f(t)$ is the nonnegative signal 1+audio, and modulate with the factor 
$$m(t) = e^{g(t)+j\mathcal{H}\{g(t)\}} $$
it can be showh that the envelope of the carrier (magnitude of the analytic carrier) will be proportional to $f^\alpha$. Since we want an envelope proportional to the square root of the audio signal, we choose $\alpha = 1/2$. This formalism reqires that the carrier is represented in the complex (analytic) domain.

In [ ]:
logsignal = np.log(1+0.95*audio)/2
explog = np.exp(analytic(logsignal))
modulated_exponential = signal.sosfilt(bpf, (explog*carrier).real)
modulated_exponential_spectrum = fft(modulated_exponential)/nsamples
demodulated_exponential = np.abs(analytic(modulated_exponential))**2
demodulated_exponential_spectrum = fft(demodulated_exponential)/nsamples

plt.plot(f,db(modulated_exponential_spectrum))
plt.xlim((30e3, 50e3))
plt.ylim((-60,0))
plt.show()
plt.plot(f,db(demodulated_exponential_spectrum))
plt.xlim((0,5e3))
plt.ylim((-60,0))
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 2, sharey=True, sharex='col')
fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0.05)

ax[0,0].plot(f,db(modulated_simple_spectrum))
ax[0,1].plot(f,db(demodulated_simple_spectrum))
ax[1][0].plot(f,db(modulated_sqrt_spectrum))
ax[1][1].plot(f,db(demodulated_sqrt_spectrum))
ax[2][0].plot(f,db(modulated_exponential_spectrum))
ax[2][1].plot(f,db(demodulated_exponential_spectrum))

#plt.setp(ax[0,0].get_xticklabels(), visible=False)

ax[0,0].set_title('Transducer signal')
ax[0,1].set_title('Audible signal')

ax[0,0].set_xlim((35e3,45e3))
#ax[0,0].set_ylim((0, 65))
ax[0,1].set_xlim((10, 5e3))
ax[0,1].set_ylim((-60, 5))

ax[0,1].set_ylabel('Simple',rotation=-90, labelpad=15)
ax[0,1].yaxis.set_label_position("right")
ax[1,1].set_ylabel('Square root',rotation=-90, labelpad=15)
ax[1,1].yaxis.set_label_position("right")
ax[2,1].set_ylabel('Exponential',rotation=-90, labelpad=15)
ax[2,1].yaxis.set_label_position("right")

ax[0,0].set_ylabel('dB')
ax[1,0].set_ylabel('dB')
ax[2,0].set_ylabel('dB')

ax[2,0].set_xlabel('Frequency in Hz')
ax[2,1].set_xlabel('Frequency in Hz')


plt.show()